In [1]:
# Check GPU availability and CUDA version
!nvidia-smi
!nvcc --version

# Import necessary modules
import os

# Set the current working directory
HOME = os.getcwd()
print("HOME:", HOME)

Thu Aug 21 20:05:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   41C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Cellpose V4

In [3]:
!pip install git+https://www.github.com/mouseland/cellpose.git

  Cloning https://www.github.com/mouseland/cellpose.git to /tmp/pip-req-build-5hgura60
  Running command git clone --filter=blob:none --quiet https://www.github.com/mouseland/cellpose.git /tmp/pip-req-build-5hgura60
  Resolved https://www.github.com/mouseland/cellpose.git to commit d95f58a6676362d4e625076e55cd73b47c3bae26
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 28.0 MB/s eta 0:00:00
  Created wheel for cellpose: filename=cellpose-4.0.7.dev14+gd95f58a66-py3-none-any.whl size=212496 sha256=1746756dfaff3dd3677103bce90013ef53843787ce87838fe685527e705d1779
  Stored in directory: /tmp/pip-ephem-wheel-cache-mtvijbzz/wheels/df/b6/31/a3013c44290eabb46f4c06d1efb19744124fcad2d59684ec5e
Successfully built cellpose


In [5]:
!pip install optuna

## Input batches

In [6]:
a_input_folders = ['/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/1. PSD95/1. Original File Names/',
                '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/2. SY38/1. Original File Names/',
               '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/3. MXO4/1. Original File Names/',
               '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/4. GFAP/1. Original File Names/'
               ]


a_gt_folders = [
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/1. PSD95/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/2. SY38/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/3. MXO4/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/4. GFAP/2. Decrypted File Names/']

b_input_folders = ['/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/1. PSD95/1. Original File Names/',
                '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/2. SY38/1. Original File Names/',
               '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/3. MXO4/1. Original File Names/',
               '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/4. GFAP/1. Original File Names/'
               ]


b_gt_folders = [
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/1. PSD95/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/2. SY38/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/3. MXO4/2. Decrypted File Names/',
            '/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/4. GFAP/2. Decrypted File Names/']

## Pretrained model paths

In [7]:
mxo4_model_paths = ['/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/models/Cellpose/MXO4_train_test_same_batch_101_tune',
'/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/models/Cellpose/MXO4_train_test_diff_batch_101_tune',
'/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/models/Cellpose/MXO4_train_test_mixed_batch_101_tune']


gfap_model_paths = ['/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/models/Cellpose/GFAP_train_test_same_batch_101_tune',
'/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/models/Cellpose/GFAP_train_test_diff_batch_101_tune',
'/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/models/Cellpose/GFAP_train_test_mixed_batch_101_tune']

## Re-tune and train example

In [11]:
command = f'python "/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/model_scripts/cellpose_retraining.py" -mn GFAP -tts 0.8 -mode train_test_same_batch \
 -input1 "{b_input_folders[3]}" -gt1 "{b_gt_folders[3]}" -input2 "{a_input_folders[3]}" -gt2 "{a_gt_folders[3]}" \
 -ts 0.3 -tunep 0.4 -trainp 0.5 -testp 0.6 -numt 1 -nume 99 -run_mode train -model_path "{gfap_model_paths[0]}"'

!{command}

2025-08-21 20:44:05,086 [INFO] WRITING LOG OUTPUT TO /root/.cellpose/run.log
2025-08-21 20:44:05,088 [INFO] 
cellpose version: 	4.0.7.dev14+gd95f58a66 
platform:       	linux 
python version: 	3.12.11 
torch version:  	2.8.0+cu126
Namespace(model_name='GFAP', train_test_split=0.8, train_test_mode='train_test_same_batch', input_folders_1='/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/4. GFAP/1. Original File Names/', gt_folders_1='/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/4. GFAP/2. Decrypted File Names/', input_folders_2='/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/4. GFAP/1. Original File Names/', gt_folders_2='/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/4. GFAP/2. Decrypted File Names/', tune_split=0.3, tune_indice_prop=0.4, train_indice_prop=0.5, test_indice_prop=0.6, num_tune_trials=1, num_epoch=99, run_mode='train', pretrained_model_path='/con

## Load retrained model example

In [10]:
command = f'python "/content/drive/MyDrive/Colab Notebooks/Segmentation_Pipeline/model_scripts/cellpose_retraining.py" -mn GFAP -tts 0.8 -mode train_test_same_batch \
 -input1 "{b_input_folders[3]}" -gt1 "{b_gt_folders[3]}" -input2 "{a_input_folders[3]}" -gt2 "{a_gt_folders[3]}" \
 -ts 0.3 -tunep 1 -trainp 1 -testp 1 -numt 5 -nume 99 -run_mode eval -model_path "{gfap_model_paths[0]}"'

!{command}

2025-08-21 20:25:47,057 [INFO] WRITING LOG OUTPUT TO /root/.cellpose/run.log
2025-08-21 20:25:47,058 [INFO] 
cellpose version: 	4.0.7.dev14+gd95f58a66 
platform:       	linux 
python version: 	3.12.11 
torch version:  	2.8.0+cu126
Namespace(model_name='GFAP', train_test_split=0.8, train_test_mode='train_test_same_batch', input_folders_1='/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_input/4. GFAP/1. Original File Names/', gt_folders_1='/content/drive/MyDrive/Colab Notebooks/Lab_images/Batch_B/Segmentation_output/4. GFAP/2. Decrypted File Names/', input_folders_2='/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_input/4. GFAP/1. Original File Names/', gt_folders_2='/content/drive/MyDrive/Colab Notebooks/Lab_images/Colab_data/Segmentation_output/4. GFAP/2. Decrypted File Names/', tune_split=0.3, tune_indice_prop=1.0, train_indice_prop=1.0, test_indice_prop=1.0, num_tune_trials=5, num_epoch=99, run_mode='eval', pretrained_model_path='/cont